#### 2024/7/15

This script is used to take out the similar datetime between InSAR and GPS time-series

In [1]:
from appgeopy import *
from my_packages import *

In [3]:
pkl_fpath = r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_004\14_POST-PROCESSING\ras2pnt_MODIFIED_oChoushui_CUMDISP_LOS_mm.pkl"
cumdisp_los_df = pd.read_pickle(pkl_fpath, compression="zip")
cumdisp_los_df.head(5)

,PointKey,X_TWD97,Y_TWD97,N20220407,N20220419,N20220501,N20220513,N20220525,N20220606,N20220618,...,N20240115,N20240127,N20240208,N20240220,N20240303,N20240315,N20240327,N20240408,N20240420,geometry
0,X237393Y2705479,237393.759851,2.705479e+06,-7.6146,-9.6779,-1.5103,13.2467,4.5566,5.2099,10.6543,...,67.6839,50.4384,45.2853,65.8027,74.2053,75.3534,77.9708,81.4065,94.7549,POINT (237393.760 2705479.209)
1,X237382Y2705465,237382.784009,2.705465e+06,-7.3033,-9.4580,-1.2588,13.4363,4.8757,5.0949,10.3587,...,66.5557,49.2441,44.0067,64.5401,72.6544,74.1423,76.8493,80.4706,93.8391,POINT (237382.784 2705465.080)
2,X237405Y2705465,237405.139575,2.705465e+06,-7.4136,-9.5174,-1.1628,13.4627,4.9123,4.9551,10.9226,...,68.7627,51.4178,46.3779,66.9026,75.6598,76.5272,79.0719,82.4995,95.9623,POINT (237405.140 2705465.403)
3,X237450Y2705438,237450.254591,2.705438e+06,-7.7303,-9.9102,-1.6172,13.2474,4.3328,3.5675,9.9899,...,63.0528,45.2381,40.2910,60.8554,69.2934,70.5222,72.4235,76.0471,89.3224,POINT (237450.255 2705438.114)
4,X237472Y2705438,237472.610164,2.705438e+06,-7.3976,-9.7601,-1.2435,13.4520,5.1330,4.3768,11.3183,...,66.5965,48.2814,43.5823,64.3737,72.2750,73.5149,75.0915,78.6874,91.9702,POINT (237472.610 2705438.437)


In [4]:
datetime_colnames = pd.Series(
    ["N20220326"] + [char for char in cumdisp_los_df if char.startswith("N")]
)
cumdisp_los_datetime = datetime_colnames.apply(
    datetime_handle.convert_to_datetime
)

In [5]:
target_datetime = cumdisp_los_datetime.loc[
    cumdisp_los_datetime < pd.Timestamp("2024-01-01")
]

In [6]:
target_datetime[:5], target_datetime[-5:]

(0   2022-03-26
 1   2022-04-07
 2   2022-04-19
 3   2022-05-01
 4   2022-05-13
 dtype: datetime64[ns],
 48   2023-11-04
 49   2023-11-16
 50   2023-11-28
 51   2023-12-10
 52   2023-12-22
 dtype: datetime64[ns])

In [10]:
output_savefolder = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2023_YehDaGang\2023_NEW_20240515\20240527_PROCESSED_DATA\_MERGED\5__TimeIntersect_Modeled"
output_excel_filename = "InSAR_Modeled_GPS_Intersect_2022_2023.xlsx"
output_excel_filepath = os.path.join(output_savefolder, output_excel_filename)
output_excel_filepath

'E:\\SUBSIDENCE_PROJECT_DATA\\GPS_2023_YehDaGang\\2023_NEW_20240515\\20240527_PROCESSED_DATA\\_MERGED\\5__TimeIntersect_Modeled\\InSAR_Modeled_GPS_Intersect_2022_2023.xlsx'

In [14]:
fpath = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2023_YehDaGang\2023_NEW_20240515\20240527_PROCESSED_DATA\_MERGED\4__Modeled\GPS_2022_2023_model_v4.xlsx"
available_sheetnames = data_io.get_sheetnames(fpath)
available_sheetnames[:5]

['ANES', 'CHSG', 'CHUA', 'DNAN', 'ERLN']

In [15]:
# select_sheet = available_sheetnames[0]
for select_sheet in tqdm(available_sheetnames):
    gps_fulltime_df = pd.read_excel(
        fpath, parse_dates=[0], index_col=[0], sheet_name=select_sheet
    )
    datetime_to_extract = gps_fulltime_df.index.intersection(target_datetime)
    gps_select_data = gps_fulltime_df.loc[datetime_to_extract, :]
    gps_select_data = gps_select_data.reset_index(drop=False)
    data_io.save_df_to_excel(
        df_to_save=gps_select_data,
        filepath=output_excel_filepath,
        sheet_name=select_sheet,
        verbose=False,
    )

100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [00:24<00:00,  2.26it/s]
